In [1]:
# Importaciones
import pandas as pd
import sys
sys.path.append("../")
from src import support_open_ai as sp
from src import support_bd as bd
import os
import dotenv # type: ignore
dotenv.load_dotenv()
import re
import json

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2

In [2]:
openai_client = sp.init_openai()

In [3]:
list_my_assistants = sp.list_asisstants(openai_client)

In [4]:
my_assistant = sp.get_assistant(openai_client, os.getenv("OPENAI_API_ASSIS"))

In [5]:
thread = sp.create_thread(openai_client)

In [34]:
supabase = bd.init_conection_bd()

In [35]:
response_select = supabase.table("leads").select("*").execute()
datos_select = pd.DataFrame(response_select.data).reset_index(drop=True)
datos_select = datos_select.drop('index', axis=1)
datos_select

,nombre,apellidos,email,estudios,especialidad,ciudad,edad,telefono,sexo,motivo_compra,score
0,Maite,Palomino,aitana54@example.com,Bachillerato,Letras,Vizcaya,43,+34 900 476 714,Hombre,Fortalecimiento del perfil profesional frente ...,NaN
1,Miriam,Almagro,odalisramon@example.org,Licenciatura,Ingenieria,Granada,21,+34 800 601 433,Hombre,Capacidad de adaptación a diferentes industrias,NaN
2,Almudena,Villaverde,graciana78@example.com,Bachillerato,Informatica,Córdoba,44,+34 739 00 32 50,Hombre,Ahorrar dinero en equipos de desarrollo para p...,NaN
3,Primitivo,Saldaña,baenacipriano@example.net,Grado Superior,Informatica,Zaragoza,21,+34841 44 87 97,Hombre,Aventurarse en el mundo freelance con más conf...,NaN
4,Emiliana,Quiroga,ramirezmaribel@example.net,Licenciatura,Ingenieria,Lugo,29,+34825 29 30 83,Mujer,Versatilidad en el mercado tecnológico,NaN
...,...,...,...,...,...,...,...,...,...,...,...
978,Sergio,Jaume,bermejorafa@example.com,Grado Superior,Ingenieria,Huelva,19,+34884 73 34 94,Hombre,Comprensión integral del ciclo de desarrollo d...,NaN
979,Elodia,Vallés,ztormo@example.com,Licenciatura,Letras,Málaga,25,+34 866 16 95 61,Mujer,Disfrutar aprendiendo nuevas tecnologías const...,NaN
980,Florinda,Quiroga,inocenciopacheco@example.org,Grado Medio,Informatica,Melilla,38,+34875 899 494,Mujer,Mejora en la comunicación entre equipos front-...,NaN
981,Manuel,Aguilera,prietolucila@example.net,Máster,Informatica,Tarragona,41,+34 806 18 79 00,Hombre,Demanda laboral en constante crecimiento,NaN


In [47]:
for index, row in datos_select.iterrows():
    df1 = pd.DataFrame(row).transpose().to_markdown()
    message =f"calcula el score de: {df1}"
    assistant_response = sp.process_data(openai_client, my_assistant.id, thread.id, message)
    break


Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
se completó exitosamente.
Thread ID: thread_L6mnTzEH11UHX5cOKJk8n8N9
Run ID: run_0XgVbUv5NhSv21Kcd5fzQXcH
Run Status: completed
respuesta procesada del asistente.


In [48]:
assistant_response

'Para calcular el score de compra para el registro proporcionado, vamos a considerar los siguientes criterios basados en los campos `motivo_compra`, `estudios`, `especialidad`, `edad` y `ciudad` de la base de datos.\n\n**Datos del registro:**\n- **Nombre**: Maite\n- **Apellidos**: Palomino\n- **Email**: aitana54@example.com\n- **Estudios**: Bachillerato\n- **Especialidad**: Letras\n- **Ciudad**: Vizcaya\n- **Edad**: 43\n- **Teléfono**: +34 900 476 714\n- **Sexo**: Hombre\n- **Motivo de compra**: Fortalecimiento del perfil profesional frente a reclutadores\n\n**Análisis:**\n1. **Motivo de compra**: El fortalecimiento del perfil profesional es un motivador muy positivo, especialmente en un contexto donde la competencia laboral es alta. Esto puede dar un score alto.\n   \n2. **Estudios**: Tener un Bachillerato es un nivel educativo básico. Este campo podría tener un impacto neutral, ya que no es un título superior, pero podría ser suficiente para ciertas posiciones laborales.\n\n3. **Espe

In [6]:
# message ="calcula el score de: | | nombre | apellidos | email | estudios | especialidad | ciudad | edad | telefono | sexo | motivo_compra | |---:|:---------|:------------|:--------------------------------|:-------------|:---------------|:---------|-------:|:----------------|:-------|:-----------------------------------------| | 0 | Violeta | Téllez | fulgenciomontenegro@example.org | Bachillerato | Letras | Burgos | 31 | +34 803 299 559 | Mujer | Demanda laboral en constante crecimiento |"
# assistant_response = sp.process_data(openai_client, my_assistant.id, thread.id, message)

Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
Esperando a que se complete...
se completó exitosamente.
Thread ID: thread_L6mnTzEH11UHX5cOKJk8n8N9
Run ID: run_7FEwWxrsHdJDHOcEdFQSLA6r
Run Status: completed
respuesta procesada del asistente.


In [49]:
def extract_json(input_text):
    try:   
        match = re.search(r"```json\n(.*?)```", input_text, re.DOTALL)
        if match:
            return json.loads(match.group(1))
        else:
            return json.loads("{}")
    except:
        return json.loads("{}")

In [51]:
data = extract_json(assistant_response)
print(data)

{'nombre': 'Maite', 'apellidos': 'Palomino', 'email': 'aitana54@example.com', 'estudios': 'Bachillerato', 'especialidad': 'Letras', 'ciudad': 'Vizcaya', 'edad': '43', 'telefono': '+34 900 476 714', 'sexo': 'Hombre', 'motivo_compra': 'Fortalecimiento del perfil profesional frente a reclutadores', 'score': '66'}


In [52]:
if (data.get("email") != None):
    if (data.get("score")) != None:
        score = data.get("score")
    else:
        score = 0

    response = (
        supabase.table("leads")
        .update({"score": score})
        .eq("email", data.get("email"))
        .execute())
else:
    print("Registro no actualizado")